# CrewAI with Couchbase Short-Term Memory

This notebook demonstrates how to implement a custom storage backend for CrewAI's memory system using Couchbase and vector search. Here's a breakdown of each section:

How to run this tutorial
----------------------
This tutorial is available as a Jupyter Notebook (.ipynb file) that you can run 
interactively. You can access the original notebook here.

You can either:
- Download the notebook file and run it on [Google Colab](https://colab.research.google.com)
- Run it on your system by setting up the Python environment

Before you start
---------------

1. Create and Deploy Your Free Tier Operational cluster on [Capella](https://cloud.couchbase.com/sign-up)
   - To get started with [Couchbase Capella](https://cloud.couchbase.com), create an account and use it to deploy 
     a forever free tier operational cluster
   - This account provides you with an environment where you can explore and learn 
     about Capella with no time constraint
   - To know more, please follow the [Getting Started Guide](https://docs.couchbase.com/cloud/get-started/create-account.html)

2. Couchbase Capella Configuration
   When running Couchbase using Capella, the following prerequisites need to be met:
   - Create the database credentials to access the required bucket (Read and Write) 
     used in the application
   - Allow access to the Cluster from the IP on which the application is running by following the [Network Security documentation](https://docs.couchbase.com/cloud/security/security.html#public-access)

## Install Required Libraries
This section installs the necessary Python packages:
- `crewai`: The main CrewAI framework
- `langchain-couchbase`: LangChain integration for Couchbase
- `langchain-openai`: LangChain integration for OpenAI
- `python-dotenv`: For loading environment variables
- `couchbase`: The Couchbase Python SDK

In [1]:
%pip install --quiet crewai langchain-couchbase langchain-openai python-dotenv couchbase

Note: you may need to restart the kernel to use updated packages.


## Importing Necessary Libraries
The script starts by importing a series of libraries required for various tasks, including handling JSON, logging, time tracking, Couchbase connections, embedding generation, and dataset loading.

In [2]:
import os
import logging
import uuid
import json
import time
from datetime import timedelta
from typing import Any, Dict, List, Optional
from dotenv import load_dotenv

# CrewAI imports
from crewai.memory.storage.rag_storage import RAGStorage
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai import Agent, Crew, Task, Process

# Couchbase imports
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions
from couchbase.auth import PasswordAuthenticator
from couchbase.diagnostics import PingState, ServiceType
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## Loading Sensitive Informnation
In this section, we prompt the user to input essential configuration settings needed. These settings include sensitive information like database credentials, and specific configuration names. Instead of hardcoding these details into the script, we request the user to provide them at runtime, ensuring flexibility and security.

The script uses environment variables to store sensitive information, enhancing the overall security and maintainability of your code by avoiding hardcoded values.

In [3]:
load_dotenv()

# Verify environment variables
required_vars = ['OPENAI_API_KEY', 'CB_HOST', 'CB_USERNAME', 'CB_PASSWORD']
for var in required_vars:
    if not os.getenv(var):
        raise ValueError(f"{var} environment variable is required")

## Implement CouchbaseStorage

This section demonstrates the implementation of a custom vector storage solution using Couchbase:

In [4]:
class CouchbaseStorage(RAGStorage):
    def __init__(self, type: str, allow_reset: bool = True, embedder_config: Optional[Dict[str, Any]] = None, crew: Optional[Any] = None):
        super().__init__(type, allow_reset, embedder_config, crew)
        self._initialize_app()

    def search(self, query: str, limit: int = 3, filter: Optional[dict] = None, score_threshold: float = 0) -> List[Dict[str, Any]]:
        try:
            # Add type filter
            search_filter = {"memory_type": self.type}
            if filter:
                search_filter.update(filter)

            # Execute search
            results = self.vector_store.similarity_search_with_score(
                query,
                k=limit,
                filter=search_filter
            )
            
            # Format results and deduplicate by content
            seen_contents = set()
            formatted_results = []
            
            for i, (doc, score) in enumerate(results):
                if score >= score_threshold:
                    content = doc.page_content
                    if content not in seen_contents:
                        seen_contents.add(content)
                        formatted_results.append({
                            "id": doc.metadata.get("memory_id", str(i)),
                            "metadata": doc.metadata,
                            "context": content,
                            "score": float(score)
                        })
            return formatted_results
        except Exception as e:
            logger.error(f"Search failed: {str(e)}")
            return []

    def save(self, value: Any, metadata: Dict[str, Any]) -> None:
        try:
            # Generate unique ID
            memory_id = str(uuid.uuid4())
            timestamp = int(time.time() * 1000)
            
            # Prepare metadata
            if not metadata:
                metadata = {}
            metadata.update({
                "memory_id": memory_id,
                "memory_type": self.type,
                "timestamp": timestamp,
                "source": "crewai"
            })

            # Convert value to string if needed
            if isinstance(value, (dict, list)):
                value = json.dumps(value)
            elif not isinstance(value, str):
                value = str(value)

            # Save to vector store
            self.vector_store.add_texts(
                texts=[value],
                metadatas=[metadata],
                ids=[memory_id]
            )
        except Exception as e:
            logger.error(f"Save failed: {str(e)}")
            raise

    def reset(self) -> None:
        if not self.allow_reset:
            return

        try:
            self.cluster.query(
                f"DELETE FROM `{self.bucket_name}`.`{self.scope_name}`.`{self.collection_name}` WHERE memory_type = $type",
                type=self.type
            ).execute()
        except Exception as e:
            logger.error(f"Reset failed: {str(e)}")
            raise

    def _initialize_app(self):
        try:
            # Initialize embeddings
            if self.embedder_config and self.embedder_config.get("provider") == "openai":
                self.embeddings = OpenAIEmbeddings(
                    openai_api_key=os.getenv('OPENAI_API_KEY'),
                    model=self.embedder_config.get("config", {}).get("model", "text-embedding-3-small")
                )
            else:
                self.embeddings = OpenAIEmbeddings(
                    openai_api_key=os.getenv('OPENAI_API_KEY'),
                    model="text-embedding-3-small"
                )

            # Connect to Couchbase
            auth = PasswordAuthenticator(
                os.getenv('CB_USERNAME', ''),
                os.getenv('CB_PASSWORD', '')
            )
            options = ClusterOptions(auth)
            
            # Initialize cluster connection
            self.cluster = Cluster(os.getenv('CB_HOST', ''), options)
            self.cluster.wait_until_ready(timedelta(seconds=5))

            # Check search service
            ping_result = self.cluster.ping()
            search_available = False
            for service_type, endpoints in ping_result.endpoints.items():
                if service_type == ServiceType.Search:
                    for endpoint in endpoints:
                        if endpoint.state == PingState.OK:
                            search_available = True
                            break
                    break
            if not search_available:
                raise RuntimeError("Search/FTS service not found or not responding")
            
            # Set up storage configuration
            self.bucket_name = os.getenv('CB_BUCKET_NAME', 'vector-search-testing')
            self.scope_name = os.getenv('SCOPE_NAME', 'shared')
            self.collection_name = os.getenv('COLLECTION_NAME', 'crew')
            self.index_name = os.getenv('INDEX_NAME', 'vector_search_crew')

            # Initialize vector store
            self.vector_store = CouchbaseVectorStore(
                cluster=self.cluster,
                bucket_name=self.bucket_name,
                scope_name=self.scope_name,
                collection_name=self.collection_name,
                embedding=self.embeddings,
                index_name=self.index_name
            )
        except Exception as e:
            logger.error(f"Initialization failed: {str(e)}")
            raise

## Test Basic Storage

Test storing and retrieving a simple memory:

In [5]:
# Initialize storage
storage = CouchbaseStorage(
    type="short_term",
    embedder_config={
        "provider": "openai",
        "config": {"model": "text-embedding-3-small"}
    }
)

# Reset storage
storage.reset()

# Test storage
test_memory = "Vector search enables semantic similarity matching by converting data into high-dimensional vectors"
test_metadata = {"category": "technology"}
storage.save(test_memory, test_metadata)

# Test search
results = storage.search("What is vector search?", limit=1)
for result in results:
    print(f"Found: {result['context']}\nScore: {result['score']}\nMetadata: {result['metadata']}")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Found: Vector search enables semantic similarity matching by converting data into high-dimensional vectors
Score: 0.6105956435203552
Metadata: {}


## Test CrewAI Integration

Create agents and tasks to test memory retention:

In [6]:
# Initialize language model
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

# Create agents
researcher = Agent(
    role='Research Expert',
    goal='Research vector search capabilities',
    backstory='Expert at finding and analyzing information about vector search technology',
    llm=llm,
    memory=True,
    memory_storage=ShortTermMemory(storage=storage)
)

writer = Agent(
    role='Technical Writer',
    goal='Create clear documentation',
    backstory='Expert at technical documentation',
    llm=llm,
    memory=True,
    memory_storage=ShortTermMemory(storage=storage)
)

# Create tasks
research_task = Task(
    description='Research vector search capabilities in modern databases. Focus on Couchbase vector search features.',
    agent=researcher,
    expected_output="A comprehensive analysis of vector search capabilities in modern databases."
)

writing_task = Task(
    description='Create documentation about vector search findings, focusing on practical implementation details.',
    agent=writer,
    context=[research_task],
    expected_output="A well-structured technical document explaining vector search implementation."
)

# Create crew
crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, writing_task],
    process=Process.sequential,
    memory=True,
    verbose=True
)

# Run crew
result = crew.kickoff()
print("\nCrew Result:\n", "-"*80)
print(result)

INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
03:26:07 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4; provider = openai


# Agent: Research Expert
## Task: Research vector search capabilities in modern databases. Focus on Couchbase vector search features.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
03:26:21 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler




# Agent: Research Expert
## Final Answer: 
Vector search, also referred to as semantic search, is an innovative search capability that employs machine learning algorithms to not just match the keywords but understand the contextual meaning of the query. This helps in providing more accurate and relevant results that are in line with the user's intent.

Vector search operates by converting the text into multi-dimensional vectors that encapsulate the semantic meaning of the text. When a search query is input, it is also converted into a vector. The database then finds and returns the vectors that are closest to the search query vector. This 'closeness' is calculated by measuring the cosine distance between vectors. This method allows the vector search to provide contextually relevant results, even if they don't contain the exact search keywords.

Couchbase, a NoSQL database, has incorporated this advanced search capability in its Full Text Search (FTS) feature. Couchbase FTS allows for

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
03:26:22 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
03:26:33 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


# Agent: Technical Writer
## Task: Create documentation about vector search findings, focusing on practical implementation details.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
03:26:51 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler




# Agent: Technical Writer
## Final Answer: 
# Vector Search Findings: Practical Implementation Details

**Introduction**

Vector search, also known as semantic search, is a leading-edge search capability that employs machine learning algorithms to understand the contextual meaning of a query. This technology enables the provision of more accurate and relevant results, aligning with users' intent.

**Conceptual Understanding of Vector Search**

Vector search turns text into multi-dimensional vectors that capture its semantic meaning. When a search query is entered, it's also converted into a vector. The database then identifies and returns the vectors that are nearest to the search query vector. This 'nearness' is calculated by measuring the cosine distance between vectors, which allows vector search to provide contextually relevant results, even if they don't contain the exact search keywords.

**Couchbase and Full Text Search (FTS)**

Couchbase, a NoSQL database, has embedded this a

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
03:26:52 - LiteLLM:INFO: utils.py:2896 - 
LiteLLM completion() model= gpt-4; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
03:27:03 - LiteLLM:INFO: utils.py:1084 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"



Crew Result:
 --------------------------------------------------------------------------------
# Vector Search Findings: Practical Implementation Details

**Introduction**

Vector search, also known as semantic search, is a leading-edge search capability that employs machine learning algorithms to understand the contextual meaning of a query. This technology enables the provision of more accurate and relevant results, aligning with users' intent.

**Conceptual Understanding of Vector Search**

Vector search turns text into multi-dimensional vectors that capture its semantic meaning. When a search query is entered, it's also converted into a vector. The database then identifies and returns the vectors that are nearest to the search query vector. This 'nearness' is calculated by measuring the cosine distance between vectors, which allows vector search to provide contextually relevant results, even if they don't contain the exact search keywords.

**Couchbase and Full Text Search (FTS)**

## Test Memory Retention

Query the stored memories to verify retention:

In [7]:
# Wait for memories to be stored
time.sleep(2)

# Query memories
memory_results = storage.search(
    query="What are the key features of vector search in Couchbase?",
    limit=2,
    score_threshold=0.0
)

print("\nMemory Search Results:\n", "-"*80)
for result in memory_results:
    print(f"Context: {result['context']}")
    print(f"Score: {result['score']}")
    print(f"Metadata: {result['metadata']}")
    print("-"*80)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"



Memory Search Results:
 --------------------------------------------------------------------------------
Context: Vector search enables semantic similarity matching by converting data into high-dimensional vectors
Score: 0.5251279473304749
Metadata: {}
--------------------------------------------------------------------------------
